In [1]:
# Imports
# pandas
import pandas as pd
from pandas import Series,DataFrame
# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

In [2]:
# Toggle based on hardware - if low memory laptop set to True else False
poor = True

(It was earlier observed that Topics of some documents are not known, Cateogires of some documents are not known, Entities of some documents are not known
)

In [ ]:
#documents_meta = pd.read_csv('./data/documents_meta.csv',  dtype={"document_id": int, "source_id": object, "publisher_id": object, "publish_time": object})

Source Id, Publisher Id and Publish time of all documents <b>are</b> known

In [73]:
#Init
events_df = pd.read_csv('./data/events.csv',  dtype={"display_id": int, "uuid": str, "document_id": int, "timestamp": int, "platform": str, "geo_location": str})
categories = pd.read_csv('./data/documents_categories.csv',  dtype={"document_id": int, "category_id": int, "confidence_level": float})

clicks_train = pd.read_csv('./data/clicks_train.csv',  dtype={"display_id": int, "ad_id": int, "clicked": int})

if poor:
    clicks_train = clicks_train.head(1001) #1002 is a different display id.

clicks_train = pd.merge(clicks_train, events_df, on='display_id')

del(events_df) # save memory

clicks_train = clicks_train.rename(index=str, columns={"document_id": "source_document_id"})

promoted_content = pd.read_csv('./data/promoted_content.csv',  dtype={"ad_id": int, "document_id": int, "campaign_id": object, "advertiser_id": object})

#(promoted_content.groupby('ad_id')['ad_id'].value_counts() != 1).unique()

clicks_train = pd.merge(clicks_train, promoted_content, on='ad_id')

del(promoted_content)

#Lets not worry about these for now
clicks_train.drop('geo_location', axis=1, inplace=True)
clicks_train.drop('platform', axis=1, inplace=True)
clicks_train.drop('timestamp', axis=1, inplace=True)
clicks_train.drop('campaign_id', axis=1, inplace=True)
clicks_train.drop('advertiser_id', axis=1, inplace=True)
clicks_train.drop('ad_id', axis=1, inplace=True)
clicks_train.drop('display_id', axis=1, inplace=True)

clicks_train.drop('uuid', axis=1, inplace=True)

Lets consider <b>Category</b> as our first feature (each Category as a unique feature)

Approach 1: Mark the document with the Category for which it has the max confidence_level and ignore other Category for that document

In [17]:
def confidence_max(topics):
    topics_grouped = topics.groupby(['document_id'], sort=False).agg({'confidence_level':'max'})
    topics_grouped = topics_grouped.reset_index()
    topics_grouped = topics_grouped.rename(columns={'confidence_level':'confidence_max'})
    topics_grouped = topics_grouped.drop_duplicates(subset=['document_id'])
    topics = pd.merge(topics, topics_grouped, how='left', on=['document_id'])
    topics_grouped = None
    topics = topics[topics['confidence_level'] == topics['confidence_max']]
    topics.drop('confidence_level', axis=1, inplace=True)
    topics.drop('confidence_max', axis=1, inplace=True)
    topics = topics.drop_duplicates(subset=['document_id'])
    return topics

In [19]:
categories = confidence_max(categories)

In [20]:
category_dummies = pd.get_dummies(categories['category_id'])

In [61]:
#category_dummies = category_dummies.mul(categories['confidence_level'], axis=0)
#categories.drop('category_id', axis=1, inplace=True) 
#categories.drop('confidence_level', axis=1, inplace=True)

In [ ]:
#clicks_train.drop('document_id', axis=1, inplace=True)

In [24]:
categories = pd.concat([categories, category_dummies], axis=1, join='inner')

In [25]:
del(category_dummies)

In [31]:
categories.iloc[2].unique()

array([1617787,       1], dtype=int64)

In [28]:
categories = categories.groupby(by='document_id', sort=False).agg(sum).reset_index() #Combine confidence level in one row

In [ ]:
#categories.to_csv('categories.csv')

In [3]:
#categories = pd.read_csv('./categories.csv',  dtype={"document_id": int})

In [11]:
#categories.drop('Unnamed: 0', axis=1, inplace=True)

In [29]:
categories = categories.to_sparse(fill_value=0)

In [33]:
clicks_train.head()

,clicked,source_document_id,document_id
0,0,379743,938164
1,0,379743,1085937
2,1,379743,1337362
3,1,1746417,1337362
4,0,442425,1337362


In [41]:
categories.head()

,document_id,1000,1100,1200,1202,1203,1204,1205,1206,1207,...,1913,1914,1915,2000,2002,2003,2004,2005,2006,2100
0,1595802,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1524246,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1617787,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1615583,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1615460,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Source merge
clicks_train = pd.merge(clicks_train, categories, how = 'left', left_on = 'source_document_id', right_on = 'document_id')

In [37]:
clicks_train.head()

,clicked,source_document_id,document_id_x,document_id_y,1000,1100,1200,1202,1203,1204,...,1913,1914,1915,2000,2002,2003,2004,2005,2006,2100
0,0,379743,938164,379743.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,379743,1085937,379743.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,379743,1337362,379743.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1746417,1337362,1746417.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,442425,1337362,442425.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
clicks_train.drop('document_id_y', axis=1, inplace=True)

clicks_train.rename(columns={'document_id_x':'document_id'}, inplace=True)

clicks_train.drop('source_document_id', axis=1, inplace=True)

In [42]:
# Dest merge
clicks_train = pd.merge(clicks_train, categories, how = 'left', left_on = 'document_id', right_on = 'document_id')

In [43]:
del(categories)

In [44]:
from sklearn import tree

model = tree.DecisionTreeClassifier(criterion='gini')

Y = clicks_train['clicked']

clicks_train.drop('clicked', axis=1, inplace=True)

clicks_train.drop('document_id', axis=1, inplace=True)

clicks_train.fillna(0, inplace=True) #NaN treated as not belonging to any Category (unknown category)

model.fit(clicks_train, Y)

In [52]:
model.score(clicks_train, Y)

0.86213786213786214

In [53]:
org_train = pd.read_csv('./data/clicks_train.csv',  dtype={"display_id": int, "ad_id": int, "clicked": int})

if poor:
    org_train = org_train.head(1001)

In [55]:
predicted = model.predict(clicks_train)

predicted[:10]

In [57]:
org_train['predicted'] = predicted

probs = model.predict_proba(clicks_train)[:,1]

org_train['probs'] = probs

org_train.sort_values(['display_id', 'probs'], inplace=True, ascending=[True, False] )

Y_ads = org_train[ org_train.clicked == 1 ].ad_id.values.reshape(-1,1)

P_ads = org_train.groupby(by='display_id', sort=False).ad_id.apply( lambda x: x.values ).values

from ml_metrics import mapk

score = mapk( Y_ads, P_ads, 12 )

In [66]:
print("MAP: %.12f" % score)

MAP: 0.502450499100


In [68]:
result = org_train.groupby(['display_id']).first()

TP = len(result[result['clicked'] == 1])

FP = len(result[result['clicked'] != 1])

print "Simple Precision = %.2f"%(TP / float(TP + FP))

Approach 2: Assign category dummy variables the actual confidence_value for that category and that document

In [ ]:
#### ------Rexecute the INIT cell

In [75]:
category_dummies = pd.get_dummies(categories['category_id'])

In [76]:
category_dummies = category_dummies.mul(categories['confidence_level'], axis=0)
categories.drop('category_id', axis=1, inplace=True) 
categories.drop('confidence_level', axis=1, inplace=True)
categories = pd.concat([categories, category_dummies], axis=1, join='inner')
del(category_dummies)
categories = categories.groupby(by='document_id', sort=False).agg(sum).reset_index() #Combine confidence level in one row
categories = categories.to_sparse(fill_value=0)
# Source merge
clicks_train = pd.merge(clicks_train, categories, how = 'left', left_on = 'source_document_id', right_on = 'document_id')
clicks_train.drop('document_id_y', axis=1, inplace=True)
clicks_train.rename(columns={'document_id_x':'document_id'}, inplace=True)
clicks_train.drop('source_document_id', axis=1, inplace=True)
clicks_train = pd.merge(clicks_train, categories, how = 'left', left_on = 'document_id', right_on = 'document_id')
del(categories)
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion='gini')
Y = clicks_train['clicked']
clicks_train.drop('clicked', axis=1, inplace=True)
clicks_train.drop('document_id', axis=1, inplace=True)
clicks_train.fillna(0, inplace=True) #NaN treated as not belonging to any Category (unknown category)
model.fit(clicks_train, Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [77]:
model.score(clicks_train, Y)

0.984015984015984

In [78]:
org_train = pd.read_csv('./data/clicks_train.csv',  dtype={"display_id": int, "ad_id": int, "clicked": int})

if poor:
    org_train = org_train.head(1001)

In [83]:
predicted = model.predict(clicks_train)
len(predicted[predicted == 1])

178

In [84]:
org_train['predicted'] = predicted

probs = model.predict_proba(clicks_train)[:,1]

org_train['probs'] = probs

org_train.sort_values(['display_id', 'probs'], inplace=True, ascending=[True, False] )

Y_ads = org_train[ org_train.clicked == 1 ].ad_id.values.reshape(-1,1)

P_ads = org_train.groupby(by='display_id', sort=False).ad_id.apply( lambda x: x.values ).values

from ml_metrics import mapk

score = mapk( Y_ads, P_ads, 12 )

In [85]:
print("MAP: %.12f" % score)

MAP: 0.490928244150


In [86]:
result = org_train.groupby(['display_id']).first()

TP = len(result[result['clicked'] == 1])

FP = len(result[result['clicked'] != 1])

print "Simple Precision = %.2f"%(TP / float(TP + FP))

Simple Precision = 0.26


Result:
1. Approach 1:Assigned document the category with which it has max confidence_level 
Result: Got MAP for 0.5 (for subset of data), Model score is 0.86 and simple precision is 0.27.
2. Approach 2:Assigned category dummy variables the actual confidence_value for that category and that document
Reuslt: Got MAP of 0.49 (for subset of data), Model score is 0.98 and simple precision is 0.27.